In [1]:
import os
import pandas as pd
from collections import defaultdict
from functools import partial
from multiprocessing import cpu_count
from pathlib import Path
from textwrap import dedent
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import Dataset
import torch as tf
from collections import Counter
from datetime import datetime
import math
from sklearn.metrics import confusion_matrix

In [2]:
def load_sensor_data_without_h(hname,lname,rname,sample_lens=36000):
    head_sensor_txt = np.genfromtxt(hname, delimiter=',', dtype=None, encoding=None)
    left_hand_sensor_txt = np.genfromtxt(lname, delimiter=',', dtype=None, encoding=None)
    right_hand_sensor_txt = np.genfromtxt(rname, delimiter=',', dtype=None, encoding=None)
    start_time = datetime.fromisoformat(head_sensor_txt[0][0]).timestamp()
    end_time = datetime.fromisoformat(head_sensor_txt[-1][0]).timestamp()
    time=end_time-start_time
    sample_timing=np.arange(0,time,time/sample_lens)
    # a 2-4 w(augular velocity) 6-8 Angle 10-12 h 14-16 q(quaternion) 18-21
    # a 0-2 w 3-5 Angle 6-8 q 9 10 11 12
    row_len = 3*3  + 1 +4
    head_data_length = len(head_sensor_txt)
    left_hand_data_length = len(left_hand_sensor_txt)
    right_hand_data_length = len(right_hand_sensor_txt) 
    total_len = 13*3
    
    head_sensor_array = np.zeros((head_data_length, row_len))
    left_hand_sensor_array = np.zeros((left_hand_data_length, row_len))
    right_hand_sensor_array = np.zeros((right_hand_data_length, row_len))
    total_sensor_array = np.zeros((len(sample_timing),total_len))
    
    for row_i, sensor_row in enumerate(head_sensor_txt):
        # a 2-4
        head_sensor_array[row_i, 0] = sensor_row[2]
        head_sensor_array[row_i, 1] = sensor_row[3]
        head_sensor_array[row_i, 2] = sensor_row[4]
        # w 6-8
        head_sensor_array[row_i, 3] = sensor_row[6]
        head_sensor_array[row_i, 4] = sensor_row[7]
        head_sensor_array[row_i, 5] = sensor_row[8]
        # Angle 10-12
        head_sensor_array[row_i, 6] = sensor_row[10]
        head_sensor_array[row_i, 7] = sensor_row[11]
        head_sensor_array[row_i, 8] = sensor_row[12]
        # q 18-21
        head_sensor_array[row_i, 9] = sensor_row[18]
        head_sensor_array[row_i, 10] = sensor_row[19]
        head_sensor_array[row_i, 11] = sensor_row[20]
        head_sensor_array[row_i, 12] = sensor_row[21]
        #time 1 
        head_sensor_array[row_i, 13] = datetime.fromisoformat(sensor_row[0]).timestamp()-start_time
        
    for row_i, sensor_row in enumerate(left_hand_sensor_txt):
        # a 2-4
        left_hand_sensor_array[row_i, 0] = sensor_row[2]
        left_hand_sensor_array[row_i, 1] = sensor_row[3]
        left_hand_sensor_array[row_i, 2] = sensor_row[4]
        # w 6-8
        left_hand_sensor_array[row_i, 3] = sensor_row[6]
        left_hand_sensor_array[row_i, 4] = sensor_row[7]
        left_hand_sensor_array[row_i, 5] = sensor_row[8]
        # Angle 10-12
        left_hand_sensor_array[row_i, 6] = sensor_row[10]
        left_hand_sensor_array[row_i, 7] = sensor_row[11]
        left_hand_sensor_array[row_i, 8] = sensor_row[12]
        # q 18-21
        left_hand_sensor_array[row_i, 9] = sensor_row[18]
        left_hand_sensor_array[row_i, 10] = sensor_row[19]
        left_hand_sensor_array[row_i, 11] = sensor_row[20]
        left_hand_sensor_array[row_i, 12] = sensor_row[21]
        #time 1 
        left_hand_sensor_array[row_i, 13] = datetime.fromisoformat(sensor_row[0]).timestamp()-start_time
    
    for row_i, sensor_row in enumerate(right_hand_sensor_txt):
        # a 2-4
        right_hand_sensor_array[row_i, 0] = sensor_row[2]
        right_hand_sensor_array[row_i, 1] = sensor_row[3]
        right_hand_sensor_array[row_i, 2] = sensor_row[4]
        # w 6-8
        right_hand_sensor_array[row_i, 3] = sensor_row[6]
        right_hand_sensor_array[row_i, 4] = sensor_row[7]
        right_hand_sensor_array[row_i, 5] = sensor_row[8]
        # Angle 10-12
        right_hand_sensor_array[row_i, 6] = sensor_row[10]
        right_hand_sensor_array[row_i, 7] = sensor_row[11]
        right_hand_sensor_array[row_i, 8] = sensor_row[12]
        # q 18-21
        right_hand_sensor_array[row_i, 9] = sensor_row[18]
        right_hand_sensor_array[row_i, 10] = sensor_row[19]
        right_hand_sensor_array[row_i, 11] = sensor_row[20]
        right_hand_sensor_array[row_i, 12] = sensor_row[21]
        #time 1 
        right_hand_sensor_array[row_i, 13] = datetime.fromisoformat(sensor_row[0]).timestamp()-start_time
    
    for i, sample_time in enumerate(sample_timing):
        idx_h = (np.abs(head_sensor_array[:,-1] - sample_time)).argmin()
        idx_l = (np.abs(left_hand_sensor_array[:,-1] - sample_time)).argmin()
        idx_r = (np.abs(right_hand_sensor_array[:,-1] - sample_time)).argmin()
        total_sensor_array[i,0:13] = head_sensor_array[idx_h,0:13]
        total_sensor_array[i,13:26] = left_hand_sensor_array[idx_l,0:13]
        total_sensor_array[i,26:] = right_hand_sensor_array[idx_r,0:13]
    return total_sensor_array

In [3]:
def compute_label(input_front, input_back, convert_dict):
    if input_front['write'] > 0.9:
        out_label = convert_dict['write']
    elif input_front['talk'] > 0.99:
        out_label = convert_dict['talk']
    elif input_front['read'] > 0.5:
        out_label = convert_dict['read']
    elif input_front['drink'] > 0.9:
        out_label = convert_dict['drink']
    elif input_front['eat'] > 0.4:
        out_label = convert_dict['eat']
    elif input_back['computer'] > 0.6:
        out_label = convert_dict['computer']
    else:
        out_label = 6
    return out_label

In [4]:
'''
def load_sensor_data_without_h(fname):
    sensor_txt = np.genfromtxt(fname, delimiter=',', dtype=None, encoding=None)
    # a 2-4 w(augular velocity) 6-8 Angle 10-12 h 14-16 q(quaternion) 18-21
    # a 0-2 w 3-5 Angle 6-8 q 9 10 11 12
    row_len = 3*3 + 4
    data_length = len(sensor_txt)
    sensor_array = np.zeros((data_length, row_len))
    for row_i, sensor_row in enumerate(sensor_txt):
        # a 2-4
        sensor_array[row_i, 0] = sensor_row[2]
        sensor_array[row_i, 1] = sensor_row[3]
        sensor_array[row_i, 2] = sensor_row[4]
        # w 6-8
        sensor_array[row_i, 3] = sensor_row[6]
        sensor_array[row_i, 4] = sensor_row[7]
        sensor_array[row_i, 5] = sensor_row[8]
        # Angle 10-12
        sensor_array[row_i, 6] = sensor_row[10]
        sensor_array[row_i, 7] = sensor_row[11]
        sensor_array[row_i, 8] = sensor_row[12]
        # q 18-21
        sensor_array[row_i, 9] = sensor_row[18]
        sensor_array[row_i, 10] = sensor_row[19]
        sensor_array[row_i, 11] = sensor_row[20]
        sensor_array[row_i, 12] = sensor_row[21]
    return sensor_array
'''
def sample_sensor_data(input_data, window_sz = 128, sample_sz = 128):
    sensor_length = input_data.shape[0]
#     print('The shape of sensor input data', input_data.shape)
    feature_sz = input_data.shape[1]
    data_sz = 0
#     print('the length of sensor', sensor_length)
    for i in range(0, sensor_length-window_sz-sample_sz, sample_sz):
        data_sz = data_sz + 1
    all_sensor_data = np.zeros((data_sz, feature_sz, window_sz))
    cnt = 0
    for i in range(0, sensor_length-window_sz-sample_sz, sample_sz):
        sample = input_data[i:i + window_sz, :]
        sample = np.transpose(sample)
        all_sensor_data[cnt, :, :] = sample
        cnt = cnt + 1
#     print('the shape of sensor dataset', all_sensor_data.shape)
    return all_sensor_data 

In [5]:
class _SepConv1d(nn.Module):
    """A simple separable convolution implementation.

    The separable convlution is a method to reduce number of the parameters
    in the deep learning network for slight decrease in predictions quality.
    """

    def __init__(self, ni, no, kernel, stride, pad):
        super().__init__()
        self.depthwise = nn.Conv1d(ni, ni, kernel, stride, padding=pad, groups=ni)
        self.pointwise = nn.Conv1d(ni, no, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))


class SepConv1d(nn.Module):
    """Implementes a 1-d convolution with 'batteries included'.

    The module adds (optionally) activation function and dropout layers right after
    a separable convolution layer.
    """

    def __init__(self, ni, no, kernel, stride, pad, drop=None,
                 activ=lambda: nn.ReLU(inplace=True)):

        super().__init__()
        assert drop is None or (0.0 < drop < 1.0)
        layers = [_SepConv1d(ni, no, kernel, stride, pad)]
        if activ:
            layers.append(activ())
        if drop is not None:
            layers.append(nn.Dropout(drop))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)


class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)


class Classifier_dh(nn.Module):
    def __init__(self, raw_ni, raw_sz, no, drop=0.05):
        super().__init__()
        self.conv1 = SepConv1d(raw_ni, 32, 8, 2, 3, drop=drop)
        self.conv2 = SepConv1d(32, 64, 8, 4, 2, drop=drop)
        self.conv3 = SepConv1d(64, 128, 8, 4, 2, drop=drop)
        self.conv4 = SepConv1d(128, 256, 8, 4, 2)
        self.flatten = Flatten()
        self.fc1 = nn.Sequential(nn.Dropout(drop), nn.Linear(raw_sz*2, 64), nn.ReLU(inplace=True))
        self.fc2 = nn.Sequential(nn.Dropout(drop), nn.Linear(64, 64), nn.ReLU(inplace=True))
        self.out = nn.Sequential(
            nn.Linear(64, 64), nn.ReLU(inplace=True), nn.Linear(64, no))

    def forward(self, t_raw):
#         print('input size', t_raw.size())
        x = self.conv1(t_raw)
#         print('conv1', x.size())
        x = self.conv2(x)
#         print('conv2', x.size())
        x = self.conv3(x)
#         print('conv3', x.size())
        x = self.conv4(x)
#         print('conv4', x.size())
        x = self.flatten(x)
#         print('flatten', x.size())
        x = self.fc1(x)
#         print('fc1', x.size())
        x = self.fc2(x)
#         print('fc2', x.size())
        x = self.out(x)
        return x

In [6]:
# Program to find most frequent  
# element in a list 
from collections import Counter
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]
def most_frequent(List):
    List = remove_values_from_list(the_list=List, val=6)
    if List == []:
        return 6
    else:
        occurence_count = Counter(List)
        return occurence_count.most_common(1)[0][0]

In [7]:
#actor 1 female:  lengths of video 1-7 are 3 minutes video 8-10 are 1 minute. video 21-26 are 3 minutes.video 27-30 are 10 minutes.
#actor 2 male: video 11-14 19 31-36 are 3 minutes
#actor 3 female: video 15-18 20 37-42 are 3 minutes 
#actor 4 female: video 43-52 are 3 minutes 
#actor 5 female: video 53-62 are 3 minutes 
#actor 6 female: 63-75
#actor 7 female: 76-85
#actor 8 female: 86-96
#actor 9 female: 97-106
#actor 10 female: 107-116
#train_num_list = [11, 12, 13, 14, 19, 15, 16, 17, 18, 20]
#val_num_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
actor_1 = [1,2,3,4,5,6,7,8,9,10]
actor_2 = [11,12,13,14,31,32,33,34,35,36]
actor_3 = [15,16,17,18,37,38,39,40,41,42]
actor_4 = [43,44,45,46,47,48,49,50,51,52]
actor_5 = [53,54,55,56,57,58,59,60,61,62]
actor_6 = [63,64,65,66,67,68,69,70,71,72]
actor_7 = [76,77,78,79,80,81,82,83,84,85]
actor_8 = [86,87,88,89,90,91,92,93,94,95]
actor_9 = [97,98,99,100,101,102,103,104,105,106]
actor_10 = [107,108,109,110,111,112,113,114,115,116]
actor_11 = [117,118,119,120,121,122,123,124,125,126]
actor_12 = [128,129,130,131,132,133,134,135,136,137]
actor_13 = [138,139,140,141,142,143,144,145,146,147]
actor_14 = [148,149,150,151,152,153,154,155,156,157]
actor_15 = [158,159,160]
actor_16 = [161,162,163,164,165,166,167,168,169,170]
actor_17 = [171,172,173,174,175,176,177,178,179,180]
actor_18 = [181,182,183,184,185,186,187,188,189,190]
actor_19 = [191,192,193,194,195,196,197,198,199,200]
actor_20 = [201,202,203,204,205,206,207,208,209,210]
actor_21 = [212,213,214,215,216,217,218,219,220,221]
actor_22 = [223,224,225,226,227,228,229,230,231,232]
actor_23 = [233,234,235,236,237,238,239,240,241,242]
actor_24 = [243,244,245,246,247,248,249,250,251,252]
actor_25 = [254,255,256,257,258,259,260,261,262,263]
actor_26 = [264,265,266,267,268,269,270,271,272,273]
train_num_list = actor_1 + actor_2 + actor_3 + actor_4 + actor_5 + actor_6 + actor_7 + actor_8 + actor_9 + actor_10 + actor_11 + actor_12 + actor_13 + actor_14+ actor_16+ actor_17+ actor_18+ actor_19+ actor_20
val_num_list = actor_21 + actor_22 + actor_23 + actor_24 + actor_25 + actor_26
#val_num_list = [38]
combine_list = train_num_list + val_num_list
print(combine_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 31, 32, 33, 34, 35, 36, 15, 16, 17, 18, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 2

In [8]:
len(combine_list)

250

In [9]:
#manual
label_array_list = {}
txt_to_label = {'talk':0, 'eat':1, 'read':2, 'drink':3, 'computer':4, 'write':5, 'other': 6}
dirpath = '/media/litao/杨力滔1/sensor_project/2020_11_24/video_sample/sensor_data_v7/'
# dirpath = '/media/rzeng/Linzh/donghao/sensorv5/video_sample/sensor_data_v7/'
#dirpath = "E:\\sensor_project\\2020_11_24\\video_sample\\sensor_data_v7\\"
manual_lable_path = dirpath + "manual_lable.csv"
manual_lable__csv = pd.read_csv(manual_lable_path)
n_lable_permin = 10000
for j in combine_list:
    
    label_array = []
    label_permin = manual_lable__csv.iloc[:,j-1]
    #print(label_permin)
    for i in range(len(label_permin)):
        if math.isnan(label_permin[i]):
            continue
        else:
            label = int(label_permin[i])
            for k in range(n_lable_permin):
                label_array.append(label)
    label_array = np.asarray(label_array)
    label_array_list[j] = label_array

In [10]:
'''
#facebook
label_array_list = {}
# train_num = 21
txt_to_label = {'talk':0, 'eat':1, 'read':2, 'drink':3, 'computer':4, 'write':5, 'other': 6}
dirpath = '/media/litao/杨力滔1/sensor_project/2020_11_24/video_sample/sensor_data_v7/'
#dirpath = "C:\\Users\\zhjsc\\Desktop\\zongyuan\\sensor\\repos\\time_series_network\\sensor_data_v7\\"

# dirpath = '/media/rzeng/Linzh/donghao/sensorv5/video_sample/sensor_data_v7/'
for j in combine_list:
    print(j)
    frontpath = dirpath + "Video" + str(j) + os.sep + "Video" + str(j) + os.sep + "front_angle.mp4_out.csv"
    front_csv = pd.read_csv(frontpath)
    backpath = dirpath + "Video" + str(j) + os.sep + "Video" + str(j) + os.sep + "back_angle.mp4_out.csv"
    back_csv = pd.read_csv(backpath)
    len_slowfast    = min(front_csv.shape[0], back_csv.shape[0])
#     print(front_csv.shape, back_csv.shape)
    label_array = []
    for i in range(len_slowfast):
        front_row = front_csv.iloc[i]
        back_row = back_csv.iloc[i]
        label = compute_label(input_front=front_row, input_back=back_row, convert_dict=txt_to_label)
        label_array.append(label)
    label_array = np.asarray(label_array)
    label_array_list[j] = label_array
'''

'\n#facebook\nlabel_array_list = {}\n# train_num = 21\ntxt_to_label = {\'talk\':0, \'eat\':1, \'read\':2, \'drink\':3, \'computer\':4, \'write\':5, \'other\': 6}\ndirpath = \'/media/litao/杨力滔1/sensor_project/2020_11_24/video_sample/sensor_data_v7/\'\n#dirpath = "C:\\Users\\zhjsc\\Desktop\\zongyuan\\sensor\\repos\\time_series_network\\sensor_data_v7\\"\n\n# dirpath = \'/media/rzeng/Linzh/donghao/sensorv5/video_sample/sensor_data_v7/\'\nfor j in combine_list:\n    print(j)\n    frontpath = dirpath + "Video" + str(j) + os.sep + "Video" + str(j) + os.sep + "front_angle.mp4_out.csv"\n    front_csv = pd.read_csv(frontpath)\n    backpath = dirpath + "Video" + str(j) + os.sep + "Video" + str(j) + os.sep + "back_angle.mp4_out.csv"\n    back_csv = pd.read_csv(backpath)\n    len_slowfast    = min(front_csv.shape[0], back_csv.shape[0])\n#     print(front_csv.shape, back_csv.shape)\n    label_array = []\n    for i in range(len_slowfast):\n        front_row = front_csv.iloc[i]\n        back_row = ba

In [10]:
label_array_list

{1: array([1, 1, 1, ..., 0, 0, 0]),
 2: array([0, 0, 0, ..., 3, 3, 3]),
 3: array([3, 3, 3, ..., 2, 2, 2]),
 4: array([4, 4, 4, ..., 1, 1, 1]),
 5: array([1, 1, 1, ..., 4, 4, 4]),
 6: array([5, 5, 5, ..., 2, 2, 2]),
 7: array([2, 2, 2, ..., 1, 1, 1]),
 8: array([1, 1, 1, ..., 2, 2, 2]),
 9: array([4, 4, 4, ..., 2, 2, 2]),
 10: array([2, 2, 2, ..., 2, 2, 2]),
 11: array([1, 1, 1, ..., 0, 0, 0]),
 12: array([3, 3, 3, ..., 2, 2, 2]),
 13: array([2, 2, 2, ..., 1, 1, 1]),
 14: array([1, 1, 1, ..., 4, 4, 4]),
 31: array([2, 2, 2, ..., 5, 5, 5]),
 32: array([4, 4, 4, ..., 3, 3, 3]),
 33: array([3, 3, 3, ..., 4, 4, 4]),
 34: array([5, 5, 5, ..., 1, 1, 1]),
 35: array([0, 0, 0, ..., 5, 5, 5]),
 36: array([0, 0, 0, ..., 1, 1, 1]),
 15: array([1, 1, 1, ..., 0, 0, 0]),
 16: array([0, 0, 0, ..., 3, 3, 3]),
 17: array([3, 3, 3, ..., 2, 2, 2]),
 18: array([4, 4, 4, ..., 1, 1, 1]),
 37: array([2, 2, 2, ..., 5, 5, 5]),
 38: array([4, 4, 4, ..., 3, 3, 3]),
 39: array([3, 3, 3, ..., 4, 4, 4]),
 40: array

In [12]:
def sensor_to_slowfast(sensor_index, sensor_data, label_data):
#     print(sensor_data.shape, label_data.shape)
    slowfast_index = int(sensor_index  / sensor_data.shape[0] * label_data.shape[0])
    return slowfast_index
window_sz = 128
sample_sz = 128
all_label_list = {}
head_sensor_data_list = {}
# train_num = 8
for j in combine_list:
    print('j', j)
    
    #frontpath = dirpath + "Video" + str(j) + os.sep + "Video" + str(j) + os.sep + "front_angle.mp4_out.csv"
    #front_csv = pd.read_csv(frontpath)
    #backpath = dirpath + "Video" + str(j) + os.sep + "Video" + str(j) + os.sep + "back_angle.mp4_out.csv"
    #back_csv = pd.read_csv(backpath)
    #len_slowfast    = min(front_csv.shape[0], back_csv.shape[0])
    len_slowfast = 30000
    
    head_sensor_name = dirpath + os.sep + "Video" + str(j) + os.sep + "Video" + str(j) + os.sep + "Head.txt"
    left_hand_sensor_name = dirpath + os.sep + "Video" + str(j) + os.sep + "Video" + str(j) + os.sep + "Left_hand.txt"
    right_hand_sensor_name = dirpath + os.sep + "Video" + str(j) + os.sep + "Video" + str(j) + os.sep + "Right_hand.txt"
    
    head_sensor = load_sensor_data_without_h(hname=head_sensor_name,lname=left_hand_sensor_name,rname=right_hand_sensor_name,sample_lens=len_slowfast)
    head_sensor_data = sample_sensor_data(head_sensor,window_sz = window_sz, sample_sz = sample_sz)
    sensor_length = head_sensor.shape[0]
    all_label = np.zeros((head_sensor_data.shape[0], 1)) 
    cnt = 0
    for k in range(0, sensor_length-window_sz-sample_sz, sample_sz):
        start = sensor_to_slowfast(sensor_index=k, sensor_data = head_sensor, label_data=label_array_list[j])
        end = sensor_to_slowfast(sensor_index=k+window_sz, sensor_data = head_sensor, label_data=label_array_list[j])
        label_data=label_array_list[j]
        cur_label_array = label_data[start:end]
        most_frequent_element = most_frequent(cur_label_array)
        all_label[cnt] = most_frequent_element
        cnt = cnt + 1
    all_label_list[j] = all_label
    head_sensor_data_list[j] = head_sensor_data

j 1
j 2
j 3
j 4
j 5
j 6
j 7
j 8
j 9
j 10
j 11
j 12
j 13
j 14
j 31
j 32
j 33
j 34
j 35
j 36
j 15
j 16
j 17
j 18
j 37
j 38
j 39
j 40
j 41
j 42
j 43
j 44
j 45
j 46
j 47
j 48
j 49
j 50
j 51
j 52
j 53
j 54
j 55
j 56
j 57
j 58
j 59
j 60
j 61
j 62
j 63
j 64
j 65
j 66
j 67
j 68
j 69
j 70
j 71
j 72
j 76
j 77
j 78
j 79
j 80
j 81
j 82
j 83
j 84
j 85
j 86
j 87
j 88
j 89
j 90
j 91
j 92
j 93
j 94
j 95
j 97
j 98
j 99
j 100
j 101
j 102
j 103
j 104
j 105
j 106
j 107
j 108
j 109
j 110
j 111
j 112
j 113
j 114
j 115
j 116
j 117
j 118
j 119
j 120
j 121
j 122
j 123
j 124
j 125
j 126
j 128
j 129
j 130
j 131
j 132
j 133
j 134
j 135
j 136
j 137
j 138
j 139
j 140
j 141
j 142
j 143
j 144
j 145
j 146
j 147
j 148
j 149
j 150
j 151
j 152
j 153
j 154
j 155
j 156
j 157
j 161
j 162
j 163
j 164
j 165
j 166
j 167
j 168
j 169
j 170
j 171
j 172
j 173
j 174
j 175
j 176
j 177
j 178
j 179
j 180
j 181
j 182
j 183
j 184
j 185
j 186
j 187
j 188
j 189
j 190
j 191
j 192
j 193
j 194
j 195
j 196
j 197
j 198
j 199
j 200
j 201
j 202


In [13]:
A = all_label_list
B = head_sensor_data_list

In [14]:
actor_1 = [1,2,3,4,5,6,7,8,9,10]
actor_2 = [11,12,13,14,31,32,33,34,35,36]
actor_3 = [15,16,17,18,37,38,39,40,41,42]
actor_4 = [43,44,45,46,47,48,49,50,51,52]
actor_5 = [53,54,55,56,57,58,59,60,61,62]
actor_6 = [63,64,65,66,67,68,69,70,71,72]
actor_7 = [76,77,78,79,80,81,82,83,84,85]
actor_8 = [86,87,88,89,90,91,92,93,94,95]
actor_9 = [97,98,99,100,101,102,103,104,105,106]
actor_10 = [107,108,109,110,111,112,113,114,115,116]
actor_11 = [117,118,119,120,121,122,123,124,125,126]
actor_12 = [128,129,130,131,132,133,134,135,136,137]
actor_13 = [138,139,140,141,142,143,144,145,146,147]
actor_14 = [148,149,150,151,152,153,154,155,156,157]
actor_15 = [161,162,163,164,165,166,167,168,169,170]
actor_16 = [171,172,173,174,175,176,177,178,179,180]
actor_17 = [181,182,183,184,185,186,187,188,189,190]
actor_18 = [191,192,193,194,195,196,197,198,199,200]
actor_19 = [201,202,203,204,205,206,207,208,209,210]
actor_20 = [212,213,214,215,216,217,218,219,220,221]
actor_21 = [223,224,225,226,227,228,229,230,231,232]
actor_22 = [233,234,235,236,237,238,239,240,241,242]
actor_23 = [243,244,245,246,247,248,249,250,251,252]
actor_24 = [254,255,256,257,258,259,260,261,262,263]
actor_25 = [264,265,266,267,268,269,270,271,272,273]

In [15]:
AAACCC=[]
CCCFFF=[]

In [16]:
for actor in range(16,25):
    print(actor)
    total_num_list = actor_1 + actor_2 + actor_3 + actor_4+actor_5+actor_6+actor_7+actor_8+actor_9+actor_10+actor_11+actor_12+actor_13+actor_14+actor_15+actor_16+actor_17+actor_18+actor_19+actor_20+actor_21+actor_22+actor_23+actor_24+actor_25
    val_num_list = total_num_list[actor*10 : actor*10+10]
    del total_num_list[actor*10 : actor*10+10]
    num_training = [5,10,15,20,24]
    for m in range(3):
        for num_train in num_training:
            train_num_list = [] 
            p=np.random.permutation(24)
            for numtrain in range(num_train):
                train_num_list+=total_num_list[p[numtrain]*10 : p[numtrain]*10+10]
            print(train_num_list)
            print(val_num_list)
            all_label_list = A
            head_sensor_data_list = B
    
    
            head_sensor_data_combine = head_sensor_data_list[train_num_list[0]]
            all_label_combine = all_label_list[train_num_list[0]]
            for train_num in train_num_list:
                head_sensor_data = head_sensor_data_list[train_num]
                head_sensor_data_combine = np.concatenate((head_sensor_data_combine,head_sensor_data), axis=0)
                all_label = all_label_list[train_num]
                all_label_combine = np.concatenate((all_label_combine, all_label), axis=0)
            cnt = 0
            all_label_combine_filtered = []
            remove_indices = []
            for i in range(len(all_label_combine)):
                if all_label_combine[i] > 5.5 and all_label_combine[i] < 6.5:
                    remove_indices.append(i)
            all_label_combine_filtered = np.delete(all_label_combine, remove_indices, axis=0)
            head_sensor_data_combine_filtered = np.delete(head_sensor_data_combine, remove_indices, axis=0)
            head_sensor_data_combine = head_sensor_data_combine_filtered
            all_label_combine = all_label_combine_filtered
            print('head_sensor dataset shape',head_sensor_data_combine.shape)
            print('head_sensor label shape', all_label_combine.shape)
    
            val_head_sensor_data = head_sensor_data_list[val_num_list[0]]
            val_label = all_label_list[val_num_list[0]]
            for val_num in val_num_list:
                head_sensor_data = head_sensor_data_list[val_num]
                val_head_sensor_data = np.concatenate((val_head_sensor_data, head_sensor_data), axis=0)
                all_label = all_label_list[val_num]
                val_label = np.concatenate((val_label, all_label), axis=0)
            remove_indices = []
            for i in range(len(val_label)):
                if val_label[i] > 5.5 and val_label[i] < 6.5:
                    remove_indices.append(i)
            val_label_filtered = np.delete(val_label, remove_indices, axis=0)
            val_head_sensor_data_filtered = np.delete(val_head_sensor_data, remove_indices, axis=0)
            val_head_sensor_data = val_head_sensor_data_filtered
            val_label = val_label_filtered
            print('validation dataset shape', val_head_sensor_data.shape)
    
            bs = 10
            sz = all_label.shape[0]
            trn_ds = TensorDataset(torch.tensor(head_sensor_data_combine).float(), torch.tensor(all_label_combine).long())
            trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, num_workers=0)
            val_label = np.asarray(val_label)
            val_ds = TensorDataset(torch.tensor(val_head_sensor_data).float(), torch.tensor(val_label).long())
            val_dl = DataLoader(val_ds, batch_size=bs, shuffle=True, num_workers=0)
    
            all_label_list = []
            val_label_list = []
            for cur_label in all_label_combine:
                all_label_list.append(cur_label[0])
            for cur_val_label in val_label:
                val_label_list.append(cur_val_label[0])
        
            lr = 0.0001
            n_epochs = 20
            iterations_per_epoch = len(trn_ds)
            num_classes = 6
            best_acc = 0
            patience, trials = 500, 0
            base = 1
            step = 2
            loss_history = []
            acc_history = []
            device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
            model = Classifier_dh(head_sensor_data.shape[1], head_sensor_data.shape[2], num_classes).to(device)
            criterion = nn.CrossEntropyLoss(reduction='sum')
            opt = optim.Adam(model.parameters(), lr=lr)
            print('Start model training')

            for epoch in range(1, n_epochs + 1):
                model.train()
                epoch_loss = 0
                for i, batch in enumerate(trn_dl):
                    x_raw, y_batch = [t.to(device) for t in batch]
                    opt.zero_grad()
                    out = model(x_raw)
                    y_batch = tf.squeeze(y_batch)
                    loss = criterion(out, y_batch)
                    epoch_loss += loss.item()
                    loss.backward()
                    opt.step()
                loss_history.append(epoch_loss)
                model.eval()
                correct, total = 0, 0
                for batch in val_dl:
                    x_raw, y_batch = [t.to(device) for t in batch]
                    y_batch = tf.squeeze(y_batch)
                    out = model(x_raw)
                    preds = F.log_softmax(out, dim=1).argmax(dim=1)

                    if preds.size()[0] > 1:
                        total += y_batch.size(0)
                        correct += (preds == y_batch).sum().item()

                acc = correct / total
                acc_history.append(acc)
                if epoch % 10 == 0:
                    print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
                if acc > best_acc:
                    trials = 0
                    best_acc = acc
                    torch.save(model.state_dict(), 'best.pth')
                    print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
                else:
                    trials += 1
                    if trials >= patience:
                        print(f'Early stopping on epoch {epoch}')
                        break
           
            model.load_state_dict(torch.load('best.pth'))
            model.eval()
    
    
            preds_list = []
            labels_list = []
            for batch in val_dl:
                x_raw, y_batch = [t.to(device) for t in batch]
                y_batch = tf.squeeze(y_batch)
                out = model(x_raw)
                preds = F.log_softmax(out, dim=1).argmax(dim=1)
                preds_list = np.concatenate((preds_list, preds.cpu().numpy()), axis=0)
                labels_list = np.concatenate((labels_list, y_batch.cpu().numpy()), axis=0)

            CCMM=confusion_matrix(y_true=labels_list, y_pred=preds_list)
            print(CCMM)
            CCCFFF.append(CCMM)
    
            TP=0
            for i in range(6):
                TP+=CCMM[i,i]
            AACC=TP/np.sum(np.sum(CCMM))
            AAACCC.append(AACC)
    
            print(AACC)

16
[201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180]
[181, 182, 183, 184, 185, 186, 187, 188, 189, 190]
head_sensor dataset shape (11883, 39, 128)
head_sensor label shape (11883, 1)
validation dataset shape (2563, 39, 128)
Start model training
Epoch 1 best model saved with accuracy: 34.57%
Epoch 2 best model saved with accuracy: 39.91%
Epoch 3 best model saved with accuracy: 46.12%
Epoch 4 best model saved with accuracy: 46.31%
Epoch 5 best model saved with accuracy: 50.14%
Epoch 8 best model saved with accuracy: 51.11%
Epoch 9 best model saved with accuracy: 51.31%
Epoch:  10. Loss: 8871.6286. Acc.: 51.42%
Epoch 10 best model saved with accuracy: 51.42%
Epoch 11 best model saved with accuracy: 53.49%
Epoch:  20. Loss: 5634.8729. Acc.: 50.21%
[[ 84  86  32  45 118  61]
 [ 31 169  66 113  21  27]
 [

head_sensor dataset shape (23533, 39, 128)
head_sensor label shape (23533, 1)
validation dataset shape (2563, 39, 128)
Start model training
Epoch 1 best model saved with accuracy: 27.58%
Epoch 2 best model saved with accuracy: 40.30%
Epoch:  10. Loss: 21725.4521. Acc.: 32.11%
Epoch:  20. Loss: 14284.4807. Acc.: 31.10%
[[ 26  38  43  38 213  68]
 [ 31  57  57 205  30  47]
 [ 11  63 115  72  96  71]
 [ 48  86  69 123  72  30]
 [  1  65  21   4 330   8]
 [  8   5  18   3   9 382]]
0.40304330862270776
[11, 12, 13, 14, 31, 32, 33, 34, 35, 36, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 15, 16, 17, 18, 37, 38, 39, 40, 41, 

head_sensor dataset shape (35183, 39, 128)
head_sensor label shape (35183, 1)
validation dataset shape (2563, 39, 128)
Start model training
Epoch 1 best model saved with accuracy: 41.36%
Epoch 2 best model saved with accuracy: 48.26%
Epoch 3 best model saved with accuracy: 48.46%
Epoch 4 best model saved with accuracy: 50.18%
Epoch 5 best model saved with accuracy: 50.25%
Epoch 9 best model saved with accuracy: 51.42%
Epoch:  10. Loss: 24473.0266. Acc.: 51.39%
Epoch 11 best model saved with accuracy: 51.93%
Epoch:  20. Loss: 16443.5962. Acc.: 51.11%
[[ 86  92  14  72 109  53]
 [ 36 164   7 169  21  30]
 [ 13 103 137  75  19  81]
 [ 48 132  21 179  10  38]
 [  8  23   4   3 375  16]
 [  9  14   7   2   3 390]]
0.51931330472103
[63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 138, 139, 140, 141, 142, 143, 144

head_sensor dataset shape (46833, 39, 128)
head_sensor label shape (46833, 1)
validation dataset shape (2563, 39, 128)
Start model training
Epoch 1 best model saved with accuracy: 47.44%
Epoch 2 best model saved with accuracy: 51.93%
Epoch 3 best model saved with accuracy: 56.54%
Epoch 4 best model saved with accuracy: 61.33%
Epoch 5 best model saved with accuracy: 62.62%
Epoch:  10. Loss: 33013.0281. Acc.: 57.86%
Epoch:  20. Loss: 22996.3612. Acc.: 60.12%
[[330  54  13  15   9   4]
 [ 67 297  22  31   7   3]
 [ 21  34 293   1  63  17]
 [ 50 250  16  54  32  24]
 [ 16   4  29   0 377   2]
 [ 29   0 118   0  27 254]]
0.6262192742879438
[86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 15, 16, 17, 18, 37, 38, 39, 40, 41, 42, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 11, 12, 13, 14, 31, 32, 33, 34, 35, 36,

head_sensor dataset shape (56153, 39, 128)
head_sensor label shape (56153, 1)
validation dataset shape (2563, 39, 128)
Start model training
Epoch 1 best model saved with accuracy: 44.28%
Epoch 2 best model saved with accuracy: 49.28%
Epoch 3 best model saved with accuracy: 50.37%
Epoch 5 best model saved with accuracy: 53.92%
Epoch 6 best model saved with accuracy: 58.21%
Epoch 7 best model saved with accuracy: 60.01%
Epoch:  10. Loss: 37895.3022. Acc.: 55.87%
Epoch:  20. Loss: 26072.9604. Acc.: 60.12%
Epoch 20 best model saved with accuracy: 60.12%
[[322  59   6  16   5  17]
 [ 80 108   6 183   4  46]
 [ 14  48 236  53  66  12]
 [ 50 103   7 213  28  25]
 [ 17  42  12   6 327  24]
 [ 19   3  17  43  11 335]]
0.6012485368708544
[107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]
[191, 192, 193, 194, 195, 196, 

...

head_sensor dataset shape (46833, 39, 128)
head_sensor label shape (46833, 1)
validation dataset shape (2563, 39, 128)
Start model training
Epoch 1 best model saved with accuracy: 25.17%
Epoch 2 best model saved with accuracy: 25.63%
Epoch 5 best model saved with accuracy: 25.71%
Epoch:  10. Loss: 32585.3968. Acc.: 24.27%
Epoch 13 best model saved with accuracy: 25.91%
Epoch 19 best model saved with accuracy: 25.95%
Epoch:  20. Loss: 22513.8962. Acc.: 24.81%
[[119  30  60   6   5 205]
 [ 25 144 118  88  19  33]
 [ 52  41 262  35   8  31]
 [ 56 174 131  27  36   1]
 [140  64 104  13  34  73]
 [251  31  39   0  29  79]]
0.259461568474444
[191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 15, 16, 17, 18, 37, 38, 39, 40, 41, 42, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 1, 2, 3, 4, 5, 6, 7, 8, 9

head_sensor dataset shape (56153, 39, 128)
head_sensor label shape (56153, 1)
validation dataset shape (2563, 39, 128)
Start model training
Epoch 1 best model saved with accuracy: 48.73%
Epoch 2 best model saved with accuracy: 49.71%
Epoch:  10. Loss: 36485.8464. Acc.: 32.81%
Epoch:  20. Loss: 24588.2317. Acc.: 41.40%
[[253  11  63  68   7  24]
 [ 17 131  23 257   0   0]
 [ 90  31 252  25   1  29]
 [ 85  62  10 269   1   1]
 [ 54  85  29 125 133   3]
 [167   8   9   4   0 236]]
0.49707374170893487
[43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137]
[254, 255, 256, 257, 258, 259, 260, 261, 262, 263]
head_sensor dataset shape (11883, 39, 128)
head_sensor label shape (11883, 1)
validation dataset shape (2563, 39, 128)
Start model training
Epoch 1 best model saved with accuracy: 30.90%
Epoch 2 best model saved with accuracy

head_sensor dataset shape (23533, 39, 128)
head_sensor label shape (23533, 1)
validation dataset shape (2563, 39, 128)
Start model training
Epoch 1 best model saved with accuracy: 28.05%
Epoch 2 best model saved with accuracy: 37.14%
Epoch 3 best model saved with accuracy: 38.51%
Epoch:  10. Loss: 14414.0201. Acc.: 30.67%
Epoch:  20. Loss: 9429.5274. Acc.: 30.04%
[[ 64  93  60  91 112   6]
 [ 32 119 126 122  26   3]
 [ 48  45 251  50  25   9]
 [ 47  90  89 149  51   2]
 [ 24 196  13  11 177   8]
 [ 52  91  32  14   8 227]]
0.3850955911041748
[15, 16, 17, 18, 37, 38, 39, 40, 41, 42, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 

head_sensor dataset shape (46833, 39, 128)
head_sensor label shape (46833, 1)
validation dataset shape (2563, 39, 128)
Start model training
Epoch 1 best model saved with accuracy: 26.88%
Epoch 2 best model saved with accuracy: 31.99%
Epoch 3 best model saved with accuracy: 32.62%
Epoch 4 best model saved with accuracy: 33.48%
Epoch:  10. Loss: 31463.7414. Acc.: 29.85%
Epoch:  20. Loss: 20722.6121. Acc.: 26.96%
[[195   7  92  17  54  60]
 [182  25  92 100   7  21]
 [ 70  23 205  50  56  25]
 [264  12  64  76   4   6]
 [ 48  15  57  14 259  35]
 [209   8  35  63  15  98]]
0.33476394849785407
[191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 63, 

Start model training
Epoch 1 best model saved with accuracy: 28.37%
Epoch:  10. Loss: 39286.5751. Acc.: 17.25%
Epoch:  20. Loss: 25400.5663. Acc.: 17.60%
[[ 41  11 105  22 120 126]
 [ 68  31 112  75  85  56]
 [ 74   5 182  29 105  34]
 [ 78   6  33  24 236  49]
 [ 76  10  37  35 245  25]
 [ 45   7  62  53  57 204]]
0.2836519703472493
[148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116]
[264, 265, 266, 267, 268, 269, 270, 271, 272, 273]
head_sensor dataset shape (11883, 39, 128)
head_sensor label shape (11883, 1)
validation dataset shape (2563, 39, 128)
Start model training
Epoch 1 best model saved with accuracy: 17.36%
Epoch 2 best model saved with accuracy: 18.53%
Epoch 3 best model saved with accuracy: 19.00%
Epoch:  10. Loss: 9579.7226. Acc.: 17.79%
Epoch:  20. Loss: 6171.3601. Acc.: 17.13%
[[ 38  23 190  49  82  43]

In [17]:
AAACCC

[0.5349200156067109,
 0.45610612563402264,
 0.4795161919625439,
 0.5255559890753024,
 0.5634022629730784,
 0.41201716738197425,
 0.40304330862270776,
 0.4760046820132657,
 0.5524775653531019,
 0.5493562231759657,
 0.23800234100663284,
 0.5017557549746391,
 0.51931330472103,
 0.5286773312524385,
 0.5637924307452205,
 0.47678501755754976,
 0.3671478735856418,
 0.5649629340616465,
 0.6262192742879438,
 0.5864221615294577,
 0.4557159578618806,
 0.40148263753413965,
 0.49707374170893487,
 0.518142801404604,
 0.6012485368708544,
 0.37339055793991416,
 0.4795161919625439,
 0.5263363246195865,
 0.5680842762387827,
 0.6414358174014826,
 0.33983612953570036,
 0.4537651190011705,
 0.277409285992977,
 0.3811939133827546,
 0.4479126024190402,
 0.31018337885290675,
 0.3187670698400312,
 0.2333203277409286,
 0.4202106905969567,
 0.44713226687475616,
 0.2629730784237222,
 0.4760046820132657,
 0.40226297307842374,
 0.41786968396410457,
 0.39875146312914556,
 0.3312524385485759,
 0.3222785797893094,
 0.

In [18]:
CCCFFF

[array([[ 84,  86,  32,  45, 118,  61],
        [ 31, 169,  66, 113,  21,  27],
        [ 15,  48, 258,   5,   5,  97],
        [ 45, 112,  31, 167,  22,  51],
        [  9,  13,  12,  10, 334,  51],
        [  3,   7,  55,   0,   1, 359]]),
 array([[ 93,  28,  10,  95,  87, 113],
        [ 25, 113,  10, 252,   0,  27],
        [ 27,  62, 108, 103,   5, 123],
        [ 36, 100,   9, 234,  10,  39],
        [  6,  71,  12,  17, 301,  22],
        [  0,  94,   5,   5,   1, 320]]),
 array([[ 76,  40,  38,  67, 154,  51],
        [ 30, 101,  43, 217,   4,  32],
        [ 69,  45, 155,  83,  11,  65],
        [ 32,  94,  42, 176,  36,  48],
        [ 11,  10,  19,  32, 345,  12],
        [ 18,  13,  16,   0,   2, 376]]),
 array([[ 60,  20,  26,  75, 126, 119],
        [ 14, 244,  70,  59,   5,  35],
        [ 53,  90, 159,  15,   5, 106],
        [ 20,  97, 114, 128,  19,  50],
        [  7,   9,  25,   4, 366,  18],
        [  8,   6,  17,   2,   2, 390]]),
 array([[ 86,  42,  23,  80, 124